In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# -------------------------------------------------
# ВСПОМОГАТЕЛЬНАЯ ФУНКЦИЯ: расчёт mAP@20
# -------------------------------------------------
def apk(actual, predicted, k=20):
    if not actual:
        return 0.0
    predicted = predicted[:k]
    score = 0.0
    hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual:
            hits += 1.0
            score += hits / (i + 1.0)
    return score / min(len(actual), k)

def map_at_k(val_interactions, user_history, top_popular, k=20, max_recent=500):
    aps = []
    for user_id, actual_set in tqdm(val_interactions.items(), desc="Оценка mAP@20"):
        pred = recommend(user_id, user_history, top_popular, top_k=k, max_recent=max_recent)
        ap = apk(actual_set, pred, k)
        aps.append(ap)
    return np.mean(aps)

# -------------------------------------------------
# ФУНКЦИЯ РЕКОМЕНДАЦИИ (без изменений)
# -------------------------------------------------
def recommend(user_id, user_history, top_popular, top_k=20, max_recent=500):
    if user_id not in user_history:
        return top_popular[:top_k]
    
    history = user_history[user_id]
    seen = set()
    recs = []
    for item in reversed(history):
        if item not in seen:
            recs.append(item)
            seen.add(item)
            if len(recs) >= max_recent:
                break
    recs = recs[:top_k]
    if len(recs) < top_k:
        for item in top_popular:
            if item not in seen:
                recs.append(item)
                if len(recs) >= top_k:
                    break
    return recs[:top_k]

# -------------------------------------------------
# 1. ЗАГРУЗКА ДАННЫХ
# -------------------------------------------------
print("📁 Загрузка данных...")
df = pd.read_parquet("train_data.pq")
df["date"] = df["date"].astype(int)
max_day_all = df["date"].max()
print(f"Всего дней в данных: {max_day_all}")

# -------------------------------------------------
# 2. ВРЕМЕННОЕ РАЗБИЕНИЕ: train / val
# -------------------------------------------------
val_days = 5  # последние 5 дней как hold-out валидация
train_end = max_day_all - val_days  # например, 47 - 5 = 42 → train: <=41

print(f"\n⏱️ Разбиение по времени:")
print(f"   Train: дни 1–{train_end}")
print(f"   Val:   дни {train_end + 1}–{max_day_all}")

train_df = df[df["date"] <= train_end].copy()
val_df = df[df["date"] > train_end].copy()

# -------------------------------------------------
# 3. ПОДГОТОВКА VAL: ground truth
# -------------------------------------------------
print("\n🔍 Формирование ground truth для валидации...")
val_interactions = val_df.groupby("user_id")["item_id"].apply(set).to_dict()
print(f"Пользователей в val: {len(val_interactions)}")

# -------------------------------------------------
# 4. ПОДГОТОВКА TRAIN: история + популярность
# -------------------------------------------------
print("\n⏳ Сбор истории пользователей (train)...")
train_df = train_df.sort_values(["user_id", "date"])
user_history_train = train_df.groupby("user_id")["item_id"].apply(list).to_dict()

print("🔥 Топ-200 популярных товаров за последние 14 дней train...")
max_day_train = train_df["date"].max()
recent_train = train_df[train_df["date"] >= (max_day_train - 13)]["item_id"]
top_popular_train = recent_train.value_counts().index[:200].tolist()

# -------------------------------------------------
# 5. ВАЛИДАЦИЯ
# -------------------------------------------------
print("\n🧪 Запуск валидации...")
map20 = map_at_k(
    val_interactions=val_interactions,
    user_history=user_history_train,
    top_popular=top_popular_train,
    k=20,
    max_recent=50
)
print(f"\n✅ mAP@20 на валидации (дни {train_end+1}–{max_day_all}): {map20:.6f}")

# -------------------------------------------------
# 6. ПОЛНАЯ МОДЕЛЬ ДЛЯ САБМИТА (обучение на всех данных)
# -------------------------------------------------
print("\n🚀 Подготовка финальной модели на всех данных...")
df_full = df.sort_values(["user_id", "date"])
user_history_full = df_full.groupby("user_id")["item_id"].apply(list).to_dict()

print("🔥 Топ-200 популярных за последние 14 дней (все данные)...")
recent_full = df_full[df_full["date"] >= (max_day_all - 13)]["item_id"]
top_popular_full = recent_full.value_counts().index[:200].tolist()

# -------------------------------------------------
# 7. ГЕНЕРАЦИЯ САБМИТА
# -------------------------------------------------
submission_template = pd.read_csv("sample_submission.csv")
unique_users = submission_template["user_id"].drop_duplicates().values
expected_total_rows = len(unique_users) * 20

print(f"\n📤 Генерация сабмита для {len(unique_users)} пользователей...")
output_file = "submission.csv"

with open(output_file, "w", encoding="utf-8") as f:
    f.write("user_id,item_id\n")
    for user_id in tqdm(unique_users, desc="Рекомендации", unit="пользователь"):
        recs = recommend(user_id, user_history_full, top_popular_full, top_k=20, max_recent=50)
        for item_id in recs:
            f.write(f"{user_id},{item_id}\n")

print(f"\n✅ Сабмит сохранён: {output_file}")
print("💡 Готов к отправке на платформу!")

📁 Загрузка данных...
Всего дней в данных: 46

⏱️ Разбиение по времени:
   Train: дни 1–41
   Val:   дни 42–46

🔍 Формирование ground truth для валидации...
Пользователей в val: 441129

⏳ Сбор истории пользователей (train)...
🔥 Топ-200 популярных товаров за последние 14 дней train...

🧪 Запуск валидации...


Оценка mAP@20: 100%|██████████| 441129/441129 [00:01<00:00, 306296.26it/s]



✅ mAP@20 на валидации (дни 42–46): 0.058704

🚀 Подготовка финальной модели на всех данных...
🔥 Топ-200 популярных за последние 14 дней (все данные)...

📤 Генерация сабмита для 293230 пользователей...


Рекомендации: 100%|██████████| 293230/293230 [00:04<00:00, 59620.38пользователь/s]


✅ Сабмит сохранён: submission.csv
💡 Готов к отправке на платформу!


In [ ]:
# 0.04143233502383816
import pandas as pd
from tqdm import tqdm

# ----------------------------
# 1. Загрузка данных
# ----------------------------
print("📁 Загрузка данных...")
train_df = pd.read_parquet("train_data.pq")
submission_template = pd.read_csv("sample_submission.csv")

# Убедимся, что дата — целое число
train_df["date"] = train_df["date"].astype(int)

# ----------------------------
# 2. Подготовка истории пользователей (сортировка по дате)
# ----------------------------
print("⏳ Сбор истории пользователей (сортировка по дате)...")
train_df = train_df.sort_values(["user_id", "date"])
user_history = train_df.groupby("user_id")["item_id"].apply(list).to_dict()

# ----------------------------
# 3. Топ-200 популярных товаров за последние 14 дней
# ----------------------------
print("🔥 Расчёт популярных товаров за последние 14 дней...")
max_day = train_df["date"].max()
recent_items = train_df[train_df["date"] >= (max_day - 13)]["item_id"]
top_popular = recent_items.value_counts().index[:200].tolist()

# ----------------------------
# 4. Функция рекомендации
# ----------------------------
def recommend(user_id, user_history, top_popular, top_k=20, max_recent=50):
    if user_id not in user_history:
        return top_popular[:top_k]
    
    history = user_history[user_id]
    seen = set()
    recs = []
    # Берём в обратном порядке — самые свежие первыми
    for item in reversed(history):
        if item not in seen:
            recs.append(item)
            seen.add(item)
            if len(recs) >= max_recent:
                break
    # Обрезаем до top_k
    recs = recs[:top_k]
    # Дополняем популярными, если не хватает
    if len(recs) < top_k:
        for item in top_popular:
            if item not in seen:
                recs.append(item)
                if len(recs) >= top_k:
                    break
    return recs[:top_k]

# ----------------------------
# 5. Получаем уникальных пользователей из шаблона
# ----------------------------
print("🔍 Определение уникальных пользователей из шаблона...")
unique_users = submission_template["user_id"].drop_duplicates().values
expected_total_rows = len(unique_users) * 20

print(f"Всего уникальных пользователей: {len(unique_users)}")
print(f"Ожидаемое число строк в сабмите: {expected_total_rows}")

# ----------------------------
# 6. Генерация сабмита — пишем напрямую в файл (экономия памяти)
# ----------------------------
output_file = "submission.csv"
print(f"📤 Генерация сабмита → {output_file} ...")

with open(output_file, "w", encoding="utf-8") as f:
    f.write("user_id,item_id\n")  # заголовок
    for user_id in tqdm(unique_users, desc="Рекомендации", unit="пользователь"):
        recs = recommend(user_id, user_history, top_popular, top_k=20, max_recent=50)
        for item_id in recs:
            f.write(f"{user_id},{item_id}\n")

print(f"\n✅ Сабмит успешно сохранён: {output_file}")
print(f"💡 Теперь можно отправлять его на платформу!")

📁 Загрузка данных...
⏳ Сбор истории пользователей (сортировка по дате)...
🔥 Расчёт популярных товаров за последние 14 дней...
🔍 Определение уникальных пользователей из шаблона...
Всего уникальных пользователей: 293230
Ожидаемое число строк в сабмите: 5864600
📤 Генерация сабмита → submission.csv ...


Рекомендации: 100%|██████████| 293230/293230 [00:07<00:00, 41491.34пользователь/s]


✅ Сабмит успешно сохранён: submission.csv
💡 Теперь можно отправлять его на платформу!
